In [1]:
from omg.datamodule import DataModule
from omg.datamodule.dataloader import OMGTorchDataset
from torch_geometric.loader import DataLoader
import torch
from omg.model.model_utils import SinusoidalTimeEmbeddings

In [2]:
dataset = DataModule("./../data/mp_20/train.lmdb", property_keys=["band_gap",])

example_data = OMGTorchDataset(dataset, convert_to_fractional=True)

loader = DataLoader(example_data, batch_size=2)

2024-09-27 13:17:03.654 | INFO     | omg.datamodule.datamodule:add_from_lmdb:832 - Reusing existing LMDB file: dataset_274518dcfedeeb30399261097eb4fcb7.lmdb


In [3]:
for batch in loader:
    t = torch.rand(1)
    print(t,batch)
    break   

/home/pawanprakash/miniconda3/envs/fermat/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


tensor([0.2341]) OMGDataBatch(
  n_atoms=[2],
  batch=[25],
  species=[25],
  cell=[2, 3, 3],
  pos=[25, 3],
  property={ band_gap=[2] },
  ptr=[3]
)


In [4]:
if bool(batch.property) == True:
    # Extract the first key-value pair dynamically
    key = list(batch.property.keys())[0]
    prop = batch.property[key]
else:
    prop = None # or prop = None is redundent here

In [5]:
print(prop.dtype, prop)

torch.float32 tensor([0.2133, 0.0000])


In [6]:
bool(batch.property)

True

In [7]:
prop = prop.double() # prop has to be same dtype as model
prop.dtype

torch.float64

In [8]:
prop_embd = SinusoidalTimeEmbeddings(32)
time_embd = SinusoidalTimeEmbeddings(64)

In [9]:
prop = prop_embd(prop)

In [10]:
prop.shape

torch.Size([2, 32])

In [11]:
t = t.repeat(2)
t.dtype

torch.float32

In [12]:
t = time_embd(t)
t.shape

torch.Size([2, 64])

In [13]:
from omg.model.encoders.cspnet_full import CSPNetFull

/home/pawanprakash/miniconda3/envs/fermat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
model = CSPNetFull(latent_dim=64)
model = model.double()

In [15]:
md = model(batch,t = t, prop = prop)

prop_indicator device:  cpu
 prop size:  torch.Size([2, 32])
 prop_indicator size:  torch.Size([2])
 node_features size:  torch.Size([25, 128])
property_indicator size:  torch.Size([2])
repeated_result size:  torch.Size([25, 128])
num atoms:  tensor([12, 13])
H_prime_L device:  cpu
H_L device:  cpu
repeated_result device:  cpu
property is here
property_indicator size:  torch.Size([2])
repeated_result size:  torch.Size([25, 128])
num atoms:  tensor([12, 13])
H_prime_L device:  cpu
H_L device:  cpu
repeated_result device:  cpu
property is here
property_indicator size:  torch.Size([2])
repeated_result size:  torch.Size([25, 128])
num atoms:  tensor([12, 13])
H_prime_L device:  cpu
H_L device:  cpu
repeated_result device:  cpu
property is here
property_indicator size:  torch.Size([2])
repeated_result size:  torch.Size([25, 128])
num atoms:  tensor([12, 13])
H_prime_L device:  cpu
H_L device:  cpu
repeated_result device:  cpu
property is here


In [16]:
md


Data(species_b=[25, 100], species_eta=[25, 100], pos_b=[25, 3], pos_eta=[25, 3], cell_b=[2, 3, 3], cell_eta=[2, 3, 3])

In [17]:
batch.species

tensor([11, 11, 11, 25, 27, 28,  8,  8,  8,  8,  8,  8, 60, 13, 13, 13, 13, 13,
        13, 13, 13, 29, 29, 29, 29])

In [18]:
md.species_b

tensor([[-0.1664,  0.0388, -0.5163,  ...,  0.3697,  0.8196, -0.1125],
        [-0.1646,  0.0390, -0.5189,  ...,  0.3694,  0.8241, -0.1150],
        [-0.1691,  0.0407, -0.5229,  ...,  0.3749,  0.8227, -0.1175],
        ...,
        [-1.0505, -0.2526, -0.0396,  ...,  1.0655,  0.6672,  0.3239],
        [-1.0544, -0.2551, -0.0410,  ...,  1.0646,  0.6673,  0.3287],
        [-1.0542, -0.2540, -0.0441,  ...,  1.0664,  0.6668,  0.3283]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)